In [ ]:
# Read data from google drive 
from google.colab import drive
drive.mount('/content/drive')

import os
# path = '/content/drive/MyDrive/projet_recherche'
path = '/content/drive/MyDrive/ENSIIE/M1/S4/PRR-Projet-Recherche/projet_recherche'
os.chdir(path)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import datetime
from functions import MAPE, MSE, MAE

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("data/param2.csv")

In [ ]:
df.shape

(100000, 12)

In [ ]:
df.head()

,S,K,m,T,r,rho,kappa,gam,nu,V0,price,impl_vol
0,100.0,109.727579,0.911348,1.289351,0.012616,-0.341331,0.466980,0.570688,0.037302,0.470395,22.052446,0.557978
1,100.0,121.761607,0.821277,1.748494,0.003466,-0.184956,0.325467,0.269769,0.112852,0.493984,25.489026,0.619066
2,100.0,75.938311,1.316858,0.718012,0.015625,-0.786272,1.965432,0.464774,0.181610,0.135622,28.586334,0.421757
3,100.0,75.997247,1.315837,1.180228,0.041085,-0.777110,1.047442,0.168897,0.027497,0.346401,34.370636,0.468116
4,100.0,101.204579,0.988098,0.699343,0.021379,-0.694197,0.622226,0.649895,0.468941,0.066212,11.764919,0.349931


# Forward Pass

### Model predicting the price

This first model predicts the price given the parameters of the Heston model. There are still a few things to check, but as a first approach it's looking good.  

In [ ]:
X = df.drop(['K', 'S', 'price', 'impl_vol'], axis = 1)
# X_stand =(X-X.mean())/X.std()
y = df['price']
# y_norm = (y-y.mean())/y.std()

In [ ]:
X.shape

(100000, 8)

In [ ]:
# The dataset should be split into 10-validation, 10-testing, 80-training 
# However, to keep thing simple, we use just a 20-80 test-train split
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# train is now 80% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

In [ ]:
# epochs is the number of iterations over the data
# verbose is a parameter to obtain additional information 
# Note: define the batch size to 
# Check if they use fully conected layers (dense) on the paper
initializer = tf.keras.initializers.GlorotUniform()
model1 = tf.keras.Sequential([
    keras.layers.Dense(units=200,input_shape=(8,),  kernel_initializer=initializer, activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=1),
    ])
model1.compile(optimizer='adam', 
               loss='mean_squared_error',
               metrics =[keras.metrics.MeanAbsoluteError(),
                         keras.metrics.MeanAbsolutePercentageError()])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1800      
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 122,601
Trainable params: 122,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = datetime.datetime.now()
print("training start time:", start)
history1= model1.fit(x_train, y_train, epochs=8000, batch_size= 1024, validation_data=(x_val, y_val), verbose=0)
end = datetime.datetime.now()
print("training end time:", end)
print('total time:', end-start)

In [ ]:
hist1 = history1.history

In [ ]:
for key in hist1.keys() :
  print(key, ':', hist1[key][8000-1])

loss : 0.001518822624348104
mean_squared_error : 0.001518822624348104
mean_absolute_error : 0.030641833320260048
mean_absolute_percentage_error : 10833.2236328125
val_loss : 0.0054627154022455215
val_mean_squared_error : 0.0054627154022455215
val_mean_absolute_error : 0.057078681886196136
val_mean_absolute_percentage_error : 26495.01953125


In [ ]:
results1 = model1.evaluate(x_test, y_test)

79/79 [==============================] - 0s 3ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - mean_absolute_error: 0.0567 - mean_absolute_percentage_error: 18037.8906


In [ ]:
for i in range(len(list(hist1.keys())[0:4])) :
  print(list(hist1.keys())[i], ':', results1[i])

loss : 0.005353164859116077
mean_squared_error : 0.005353164859116077
mean_absolute_error : 0.056735455989837646
mean_absolute_percentage_error : 18037.890625


In [ ]:
y_pred = model1.predict(x_test)

In [ ]:
y_test.head(10)

82381    42.380078
46057    16.905663
74567     1.493860
5282     29.524343
86634    43.531872
17513    29.435774
34099    27.826019
86343    43.963627
93382    20.749449
68808     1.675176
Name: price, dtype: float64

In [ ]:
y_pred[0:10]

array([[42.344017 ],
       [16.908045 ],
       [ 1.4663668],
       [29.517649 ],
       [43.42189  ],
       [29.395102 ],
       [27.795198 ],
       [43.892387 ],
       [20.660273 ],
       [ 1.6506165]], dtype=float32)

In [ ]:
MSE(y_test, y_pred)

2765960.05045

In [ ]:
MAE(y_test, y_pred)

135180.88815

In [ ]:
MAPE(y_test, y_pred)

5730024471158427.0

In [ ]:
file_name = 'model1'
pd.DataFrame(hist1).to_csv('results/'+ file_name + '_train.csv', index = False)

In [ ]:
pd.DataFrame(results1).transpose().set_axis(['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], axis=1).to_csv('results/'+ file_name +'_test.csv', index = False)

### Model predicting the implied volatility

In [ ]:
X = df.drop(['price', 'K', 'S', 'impl_vol'], axis=1)
# X_stand =(X-X.mean())/X.std()
y = df['impl_vol']
# y_norm = (y-y.mean())/y.std()

In [ ]:
X.shape

(100000, 8)

In [ ]:
# The dataset should be split into 10-validation, 10-testing, 80-training 
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# train is now 80% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

In [ ]:
# epochs is the number of iterations over the data
# verbose is a parameter to obtain additional information 
# Note: define the batch size to 
initializer = tf.keras.initializers.GlorotUniform()
model2 = tf.keras.Sequential([
    keras.layers.Dense(units=200,input_shape=(8,),  kernel_initializer=initializer, activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=1),
    ])
model2.compile(optimizer='adam', 
               loss='mean_squared_error',
               metrics =[keras.metrics.MeanAbsoluteError(),
                         keras.metrics.MeanAbsolutePercentageError()])
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1800      
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 122,601
Trainable params: 122,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = datetime.datetime.now()
print("training start time:", start)
history2= model2.fit(x_train, y_train, epochs=8000, batch_size= 1024, validation_data=(x_val, y_val), verbose=0)
end = datetime.datetime.now()
print("training end time:", end)
print('total time:', end-start)

training start time: 2022-04-23 15:06:02.316187
training end time: 2022-04-23 16:05:26.026481
total time: 0:59:23.710294


In [ ]:
hist2 = history2.history

In [ ]:
for key in hist2.keys() :
  print(key, ':', hist2[key][8000-1])

loss : 2.538328089940478e-07
mean_squared_error : 2.538328089940478e-07
mean_absolute_error : 0.0004012206627521664
mean_absolute_percentage_error : 101.51593017578125
val_loss : 4.0691334106668364e-06
val_mean_squared_error : 4.0691334106668364e-06
val_mean_absolute_error : 0.0007561574457213283
val_mean_absolute_percentage_error : 3536.4375


In [ ]:
file_name = 'model2'
pd.DataFrame(hist2).to_csv('results/'+ file_name + '_train.csv', index = False)

In [ ]:
results2 = model2.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 1.9140e-06 - mean_squared_error: 1.9140e-06 - mean_absolute_error: 7.4602e-04 - mean_absolute_percentage_error: 0.2086


In [ ]:
for i in range(len(list(hist2.keys())[0:4])) :
  print(list(hist2.keys())[i], ':', results2[i])

loss : 1.914025233418215e-06
mean_squared_error : 1.914025233418215e-06
mean_absolute_error : 0.0007460200577042997
mean_absolute_percentage_error : 0.20858025550842285


In [ ]:
pd.DataFrame(results2).transpose().set_axis(['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], axis=1).to_csv('results/'+ file_name +'_test.csv', index = False)

In [ ]:
# y_pred = model.predict(x_test)
y_pred = model2.predict(x_test)

In [ ]:
y_test.head()

82381    0.612825
46057    0.566543
74567    0.283900
5282     0.652536
86634    0.566911
Name: impl_vol, dtype: float64

In [ ]:
y_pred[0:5]

array([[0.61214507],
       [0.5652702 ],
       [0.28207326],
       [0.65184397],
       [0.5654832 ]], dtype=float32)

In [ ]:
MAPE(y_test, y_pred)

3453.58047

In [ ]:
MSE(y_test, y_pred)

290.25779

In [ ]:
MAE(y_test, y_pred)

1381.44005

# Backward Pass

Model predicting the price

In [ ]:
df.head()

,S,K,m,T,r,rho,kappa,gam,nu,V0,price,impl_vol
0,100.0,109.727579,0.911348,1.289351,0.012616,-0.341331,0.466980,0.570688,0.037302,0.470395,22.052446,0.557978
1,100.0,121.761607,0.821277,1.748494,0.003466,-0.184956,0.325467,0.269769,0.112852,0.493984,25.489026,0.619066
2,100.0,75.938311,1.316858,0.718012,0.015625,-0.786272,1.965432,0.464774,0.181610,0.135622,28.586334,0.421757
3,100.0,75.997247,1.315837,1.180228,0.041085,-0.777110,1.047442,0.168897,0.027497,0.346401,34.370636,0.468116
4,100.0,101.204579,0.988098,0.699343,0.021379,-0.694197,0.622226,0.649895,0.468941,0.066212,11.764919,0.349931


In [ ]:
# X = df.drop(['K', 'S', 'price', 'impl_vol'], axis = 1)
X = df[['m', 'T', 'r', 'price']]
y = df[['rho',	'kappa',	'gam',	'nu',	'V0']]

In [ ]:
print(X.shape)
X.head()

(100000, 4)


,m,T,r,price
0,0.911348,1.289351,0.012616,22.052446
1,0.821277,1.748494,0.003466,25.489026
2,1.316858,0.718012,0.015625,28.586334
3,1.315837,1.180228,0.041085,34.370636
4,0.988098,0.699343,0.021379,11.764919


In [ ]:
print(y.shape)
y.head()

(100000, 5)


,rho,kappa,gam,nu,V0
0,-0.341331,0.466980,0.570688,0.037302,0.470395
1,-0.184956,0.325467,0.269769,0.112852,0.493984
2,-0.786272,1.965432,0.464774,0.181610,0.135622
3,-0.777110,1.047442,0.168897,0.027497,0.346401
4,-0.694197,0.622226,0.649895,0.468941,0.066212


In [ ]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# train is now 80% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

In [ ]:
# epochs is the number of iterat ions over the data
# verbose is a parameter to obtain additional information 
# Note: define the batch size to 
initializer = tf.keras.initializers.GlorotUniform()
model3 = tf.keras.Sequential([
    keras.layers.Dense(units=200,input_shape=(4,),  kernel_initializer=initializer, activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=5),
    ])
model3.compile(optimizer='adam', 
               loss='mean_squared_error',
               metrics =[keras.metrics.MeanAbsoluteError(),
                         keras.metrics.MeanAbsolutePercentageError()])
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1000      
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dense_4 (Dense)             (None, 5)                 1005      
                                                                 
Total params: 122,605
Trainable params: 122,605
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = datetime.datetime.now()
print("training start time:", start)
history3 = model3.fit(x_train, y_train, epochs=8000, batch_size= 1024, validation_data=(x_val, y_val), verbose=0)
end = datetime.datetime.now()
print("training end time:", end)
print('total time:', end-start)

training start time: 2022-05-01 10:35:06.753583
training end time: 2022-05-01 11:33:55.395046
total time: 0:58:48.641463


In [ ]:
hist3 = history3.history

In [ ]:
for key in hist3.keys() :
  print(key, ':', hist3[key][8000-1])

loss : 0.09046310186386108
mean_absolute_error : 0.197271466255188
mean_absolute_percentage_error : 375.5846862792969
val_loss : 0.21235810220241547
val_mean_absolute_error : 0.26035675406455994
val_mean_absolute_percentage_error : 329.8963928222656


In [ ]:
file_name = 'model3'
pd.DataFrame(hist3).to_csv('results/'+ file_name + '_train.csv', index = False)

In [ ]:
results3 = model3.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.2168 - mean_absolute_error: 0.2606 - mean_absolute_percentage_error: 345.5096


In [ ]:
hist3.keys()

dict_keys(['loss', 'mean_absolute_error', 'mean_absolute_percentage_error', 'val_loss', 'val_mean_absolute_error', 'val_mean_absolute_percentage_error'])

In [ ]:
for i in range(len(list(hist3.keys())[0:3])) :
  print(list(hist3.keys())[i], ':', results3[i])

loss : 0.21678923070430756
mean_absolute_error : 0.2606263756752014
mean_absolute_percentage_error : 345.5096435546875


In [ ]:
pd.DataFrame(results3).transpose().set_axis(['loss', 'mean_absolute_error', 'mean_absolute_percentage_error'], axis=1).to_csv('results/'+ file_name +'_test.csv', index = False)

In [ ]:
y_pred = model3.predict(x_test)

In [ ]:
y_pred[0:5]

array([[-0.60211635,  0.48164433,  0.48363024,  0.2684998 ,  0.42852208],
       [-0.47623584,  0.98794943,  0.53052425,  0.27662578,  0.38817495],
       [-0.38112855,  2.469232  ,  0.5739757 ,  0.12130196,  0.09638207],
       [-0.47136182,  0.54197663,  0.470691  ,  0.34745857,  0.45442346],
       [-0.53230953,  0.27428943,  0.37134108,  0.2531854 ,  0.41938442]],
      dtype=float32)

In [ ]:
y_test

,rho,kappa,gam,nu,V0
82381,-0.067017,0.116469,0.208811,0.470432,0.371684
46057,-0.814302,0.731195,0.178829,0.187846,0.392124
74567,-0.844973,1.423656,0.699284,0.218922,0.093206
5282,-0.185256,0.019181,0.634748,0.107740,0.463973
86634,-0.455773,0.133045,0.367305,0.318220,0.375766
...,...,...,...,...,...
6086,-0.644552,0.960693,0.283136,0.136992,0.482355
12619,-0.838358,0.071290,0.514188,0.199272,0.447563
51406,-0.452403,2.058821,0.635629,0.208253,0.137684
45311,-0.597482,1.560774,0.402551,0.148438,0.287744


Model predicting the implied volatility

In [ ]:
df.head()

,S,K,m,T,r,rho,kappa,gam,nu,V0,price,impl_vol
0,100.0,109.727579,0.911348,1.289351,0.012616,-0.341331,0.466980,0.570688,0.037302,0.470395,22.052446,0.557978
1,100.0,121.761607,0.821277,1.748494,0.003466,-0.184956,0.325467,0.269769,0.112852,0.493984,25.489026,0.619066
2,100.0,75.938311,1.316858,0.718012,0.015625,-0.786272,1.965432,0.464774,0.181610,0.135622,28.586334,0.421757
3,100.0,75.997247,1.315837,1.180228,0.041085,-0.777110,1.047442,0.168897,0.027497,0.346401,34.370636,0.468116
4,100.0,101.204579,0.988098,0.699343,0.021379,-0.694197,0.622226,0.649895,0.468941,0.066212,11.764919,0.349931


In [ ]:
X = df[['m', 'T', 'r', 'impl_vol']]
y = df[['rho',	'kappa',	'gam',	'nu',	'V0']]

In [ ]:
print(X.shape)
X.head()

(100000, 4)


,m,T,r,impl_vol
0,0.911348,1.289351,0.012616,0.557978
1,0.821277,1.748494,0.003466,0.619066
2,1.316858,0.718012,0.015625,0.421757
3,1.315837,1.180228,0.041085,0.468116
4,0.988098,0.699343,0.021379,0.349931


In [ ]:
print(y.shape)
y.head()

(100000, 5)


,rho,kappa,gam,nu,V0
0,-0.341331,0.466980,0.570688,0.037302,0.470395
1,-0.184956,0.325467,0.269769,0.112852,0.493984
2,-0.786272,1.965432,0.464774,0.181610,0.135622
3,-0.777110,1.047442,0.168897,0.027497,0.346401
4,-0.694197,0.622226,0.649895,0.468941,0.066212


In [ ]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# train is now 80% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

In [ ]:
# epochs is the number of iterat ions over the data
# verbose is a parameter to obtain additional information 
# Note: define the batch size to 
initializer = tf.keras.initializers.GlorotUniform()
model4 = tf.keras.Sequential([
    keras.layers.Dense(units=200,input_shape=(4,),  kernel_initializer=initializer, activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=5),
    ])
model4.compile(optimizer='adam', 
               loss='mean_squared_error',
               metrics =[keras.metrics.MeanAbsoluteError(),
                         keras.metrics.MeanAbsolutePercentageError()])
model4.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 200)               1000      
                                                                 
 dense_6 (Dense)             (None, 200)               40200     
                                                                 
 dense_7 (Dense)             (None, 200)               40200     
                                                                 
 dense_8 (Dense)             (None, 200)               40200     
                                                                 
 dense_9 (Dense)             (None, 5)                 1005      
                                                                 
Total params: 122,605
Trainable params: 122,605
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = datetime.datetime.now()
print("training start time:", start)
history4 = model4.fit(x_train, y_train, epochs=8000, batch_size= 1024, validation_data=(x_val, y_val), verbose=0)
end = datetime.datetime.now()
print("training end time:", end)
print('total time:', end-start)

training start time: 2022-05-01 12:06:51.329975
training end time: 2022-05-01 13:07:09.619386
total time: 1:00:18.289411


In [ ]:
hist4 = history4.history

In [ ]:
for key in hist4.keys() :
  print(key, ':', hist3[key][8000-1])

loss : 0.09046310186386108
mean_absolute_error : 0.197271466255188
mean_absolute_percentage_error : 375.5846862792969
val_loss : 0.21235810220241547
val_mean_absolute_error : 0.26035675406455994
val_mean_absolute_percentage_error : 329.8963928222656


In [ ]:
file_name = 'model4'
pd.DataFrame(hist4).to_csv('results/'+ file_name + '_train.csv', index = False)

In [ ]:
results4 = model4.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.2227 - mean_absolute_error: 0.2635 - mean_absolute_percentage_error: 347.0923


In [ ]:
for i in range(len(list(hist4.keys())[0:3])) :
  print(list(hist4.keys())[i], ':', results3[i])

loss : 0.21678923070430756
mean_absolute_error : 0.2606263756752014
mean_absolute_percentage_error : 345.5096435546875


In [ ]:
pd.DataFrame(results4).transpose().set_axis(['loss', 'mean_absolute_error', 'mean_absolute_percentage_error'], axis=1).to_csv('results/'+ file_name +'_test.csv', index = False)

In [ ]:
y_pred = model4.predict(x_test)

In [ ]:
y_pred[0:5]

array([[-0.52178997,  0.4511963 ,  0.46107525,  0.29601914,  0.45007932],
       [-0.49531314,  0.9808105 ,  0.49921322,  0.24128503,  0.38836002],
       [-0.6733142 ,  2.6027126 ,  0.59779036,  0.31973436,  0.1476802 ],
       [-0.4705751 ,  0.2219376 ,  0.46256483,  0.3106338 ,  0.44276756],
       [-0.55294997,  0.24533635,  0.41397905,  0.25299633,  0.4057047 ]],
      dtype=float32)

In [ ]:
y_test.head()

,rho,kappa,gam,nu,V0
82381,-0.067017,0.116469,0.208811,0.470432,0.371684
46057,-0.814302,0.731195,0.178829,0.187846,0.392124
74567,-0.844973,1.423656,0.699284,0.218922,0.093206
5282,-0.185256,0.019181,0.634748,0.107740,0.463973
86634,-0.455773,0.133045,0.367305,0.318220,0.375766
